# NLP to SQL Project

In this project, we'll develop a start-up that can take a non-technical manager's Natural Language question, such as "What counties had the top sales?" and convert that into a SQL query. We will then use that SQL to query the data (which in this example will come from a .csv file we read in with Pandas and set-up a temporary DB in RAM) and then report back the results!

## Imports

In [2]:
# !pip install openai

In [3]:
import os
import pandas as pd
#pip install pandas

## Data


Our data will be some example Sales Data from:  https://www.kaggle.com/datasets/kyanyoga/sample-sales-data

Let's read it in:

In [4]:
df = pd.read_csv("sales_data_sample.csv")
#read sales data smaple file csv

In [5]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


## Querying Data

We can query via Pandas Syntax:

In [38]:
# Example Pandas Query for Sum of Sales by Quarter 
df.groupby("QTR_ID").sum()['SALES']
# query without using openai

QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64


Or we can query via SQL Syntax, let's set-up a temporary in memory (RAM) database, basically export this CSV-->Pandas DF--> SQL DB

## SQL Database Set-up

In [39]:
from sqlalchemy import create_engine
from sqlalchemy import text
# connect to different database engine

In [40]:
# setup a temp database in ram

In [41]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

Push our entire DataFrame to a table called Sales:

In [42]:
data = df.to_sql(name='Sales',con=temp_db)
# push pandas dataframe into temp database
# name and con(connection to what db)

2024-05-31 13:39:51,800 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2024-05-31 13:39:51,801 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-31 13:39:51,803 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2024-05-31 13:39:51,804 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-31 13:39:51,808 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-31 13:39:51,811 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20


### Connecting to SQL Database:

Using SQL Alchemy we can establish a connection to this temporary database and query it for the results:

In [43]:
with temp_db.connect() as conn:
    result = conn.execute(text("Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1"))
# make the connection
# run code
# text call to pass thr sql query and execute the sql query to the db
# close connection

2024-05-31 13:39:56,315 INFO sqlalchemy.engine.Engine Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1
2024-05-31 13:39:56,317 INFO sqlalchemy.engine.Engine [generated in 0.00198s] ()


In [44]:
result.all()

[(10407, 14082.8)]

## OpenAI API

Set your OpenAI API Key as an environment variable. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

### Set-up Open AI API Key

In [45]:
import openai

### Inform GPT about the SQL Table Structure

We need to tell GPT what the table structure looks like before it can understand the schema enough to create a SQL query. Let's create a function to generate the first part of the example text below (which we can then attach a user natural language query to!)


**Below is an example input to GPT, we tell it the table structure and the NLP question, then at the end we tell it to "SELECT"...**

**Thus GPT must finish with the rest of the most reasonable SQL query.**

In [46]:
### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months
# SELECT


In [47]:
def create_table_definition_prompt(df):
    """
    This function returns a prompt that informs GPT that we want to work with SQL Tables and what the overall goal is
    """

    prompt = '''Given the following sqlite SQL definition, write queries based on the request 
                \n### sqlite SQL table, with its properties:
#
# Sales({})
#
'''.format(",".join(str(x) for x in df.columns))
    
    return prompt

In [48]:
print(create_table_definition_prompt(df))

Given the following sqlite SQL definition, write queries based on the request 
                
### sqlite SQL table, with its properties:
#
# Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
#



### Get Natural Language Request:

Now let's create a function that grabs the natural language information request. Users can input any instructions they want to do with this dataset

In [49]:
def prompt_input():
    nlp_text = input("Enter information you want to obtain: ")
    return nlp_text

In [51]:
nlp_text = prompt_input()

Enter information you want to obtain: find the average of sales per location


In [52]:
nlp_text

'find the average of sales per location'

Now we call the model:

### OpenAI API Call

In [53]:
#pip install openai==0.28
# degrade to gpt3.5 model for performance

In [72]:
# Set your OpenAI API key
api_key = 

# Configure the OpenAI client with your API key
openai.api_key = api_key

# Assuming create_table_definition_prompt and df are defined earlier in your script
# Also assuming nlp_text is defined earlier in your script

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": create_table_definition_prompt(df)},
    {"role": "user", "content": f"A query to answer: {nlp_text}"},
  ]
)

print(response)

{
  "id": "chatcmpl-9V56zjVsfWn4k7j0fSzHvJQUqGpks",
  "object": "chat.completion",
  "created": 1717194845,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "```sql\nSELECT CITY, STATE, COUNTRY, TERRITORY, AVG(SALES) AS AVERAGE_SALES\nFROM Sales\nGROUP BY CITY, STATE, COUNTRY, TERRITORY;\n```"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 135,
    "completion_tokens": 44,
    "total_tokens": 179
  },
  "system_fingerprint": null
}


In [73]:
response['choices'][0]['message']['content']

'```sql\nSELECT CITY, STATE, COUNTRY, TERRITORY, AVG(SALES) AS AVERAGE_SALES\nFROM Sales\nGROUP BY CITY, STATE, COUNTRY, TERRITORY;\n```'

Now let's build a function to parse the section of the response we want: 

In [74]:
print(response)

{
  "id": "chatcmpl-9V56zjVsfWn4k7j0fSzHvJQUqGpks",
  "object": "chat.completion",
  "created": 1717194845,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "```sql\nSELECT CITY, STATE, COUNTRY, TERRITORY, AVG(SALES) AS AVERAGE_SALES\nFROM Sales\nGROUP BY CITY, STATE, COUNTRY, TERRITORY;\n```"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 135,
    "completion_tokens": 44,
    "total_tokens": 179
  },
  "system_fingerprint": null
}


In [75]:
def handle_response(response):
    return response.choices[0].message.content

In [76]:
print(type(handle_response(response)))
print(handle_response(response))
#we need to cut the unecessary part of the returning string

returntype = handle_response(response)[6:len(handle_response(response))-5]
print(returntype)


<class 'str'>
```sql
SELECT CITY, STATE, COUNTRY, TERRITORY, AVG(SALES) AS AVERAGE_SALES
FROM Sales
GROUP BY CITY, STATE, COUNTRY, TERRITORY;
```

SELECT CITY, STATE, COUNTRY, TERRITORY, AVG(SALES) AS AVERAGE_SALES
FROM Sales
GROUP BY CITY, STATE, COUNTRY, TERRITORY


Pass that into our Database:

In [77]:
with temp_db.connect() as conn:
    result = conn.execute(text(returntype))

2024-05-31 15:34:51,317 INFO sqlalchemy.engine.Engine 
SELECT CITY, STATE, COUNTRY, TERRITORY, AVG(SALES) AS AVERAGE_SALES
FROM Sales
GROUP BY CITY, STATE, COUNTRY, TERRITORY
2024-05-31 15:34:51,319 INFO sqlalchemy.engine.Engine [generated in 0.00266s] ()


In [63]:
result.all()

[('Aaarhus', None, 'Denmark', 'EMEA', 3725.7611111111105),
 ('Allentown', 'PA', 'USA', None, 3939.9400000000005),
 ('Barcelona', None, 'Spain', 'EMEA', 3409.2113043478266),
 ('Bergamo', None, 'Italy', 'EMEA', 2874.077500000001),
 ('Bergen', None, 'Norway', 'EMEA', 3849.6648275862067),
 ('Boras', None, 'Sweden', 'EMEA', 3533.1402631578953),
 ('Boston', 'MA', 'USA', None, 3501.5831818181814),
 ('Brickhaven', 'MA', 'USA', None, 3516.068085106384),
 ('Bridgewater', 'CT', 'USA', None, 4075.7916000000005),
 ('Brisbane', 'CA', 'USA', None, 3347.9006666666673),
 ('Bruxelles', None, 'Belgium', 'EMEA', 2998.9008000000003),
 ('Burbank', 'CA', 'USA', None, 3544.972307692307),
 ('Burlingame', 'CA', 'USA', None, 3552.443235294118),
 ('Cambridge', 'MA', 'USA', None, 3664.315789473683),
 ('Charleroi', None, 'Belgium', 'EMEA', 4180.0125),
 ('Chatswood', 'NSW', 'Australia', 'APAC', 3295.021304347827),
 ('Cowes', 'Isle of Wight', 'UK', 'EMEA', 3009.2630769230764),
 ('Dublin', None, 'Ireland', 'EMEA', 360